In [1]:
# Imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from unidecode import unidecode

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [2]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

review  polarity
0      Ça commence à devenir énervant d'avoir l'impre...         0
1      J'ai aimé ce film, si il ressemble a un docume...         1
2      Une grosse merde ce haneke ce faire produire p...         0
3      Beau mélodrame magnifiquement photographié, "V...         1
4      A la poursuite du diamant vers est un film pro...         1
...                                                  ...       ...
29946  Le meilleur film de super-héros derrière le ba...         1
29947  Un drame qui est d'une efficacité remarquable....         1
29948  Une daube hollywoodienne de plus, aucun intérê...         0
29949  Et voilà un nouveau biopic sur la star du X Li...         0
29950  Un film qui fait vieux, avec des acteurs pas t...         0

[29951 rows x 2 columns]

In [3]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [4]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [5]:
# Will be used to eliminate non a-z characters
def filter_out_non_letters(text):
    
    new_text = ''
    
    for character in text:
        
        if character.isalpha() or character == ' ':
            new_text += character
    
    return new_text

In [6]:
%%time

# We eliminate accents 
X_clean = X.map(unidecode)

# We lower all letters
X_clean = X_clean.map(lambda string: string.lower())

# We only keep a-z characters
X_clean = X_clean.map(filter_out_non_letters)

CPU times: user 9.92 s, sys: 42.5 ms, total: 9.96 s
Wall time: 10 s


In [7]:
# Let us look at the result for the first review
X_clean[0]

'ca commence a devenir enervant davoir limpression de voir et revoir le meme genre de film a savoir  la comedie romantique surement le genre le plus prolifique de le production francaise actuelle le probleme cest que lon a souvent affaire a des niaiseries de faible niveau comme celui ci avec un scenario ultra balise et conventionnel cest a se demander comment ca peut passer les portes dun producteur bref cette sempiternel histoire dun homme mentant au nom de lamour pour reconquerir une femme et qui a la fin se prend son mensonge en pleine figure est dune originalite affligeante et ce nest pas la presence au casting de lex miss meteo charlotte le bon qui reve surement davoir la meme carriere que louise bourgoin qui change la donne'

In [8]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [9]:
# We start from the clean solution
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [10]:
# We instantiate a tokenizer
tokenizer = Tokenizer()

# Which we fit on the text
tokenizer.fit_on_texts(X_clean)

In [11]:
# We output sequences of integers (one integer per word) thanks to the tokenizer
sequences = tokenizer.texts_to_sequences(X_clean)

# We pad the sequences (or truncate them) to the right so that they all gather 100 integers
sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [12]:
# We move the data to a NumPy array
X_input = np.array(sequences)

# Whose shape we can print
print(X_input.shape)

# And we can have a look at the first sequence
X_input[0]

(29951, 100)


array([   47,   458,     3,   739,  4022,     1,    51,     2,   390,
           4,    37,   100,     1,     7,     3,   373,     5,    99,
         655,   781,     4,   100,     4,    23,  8891,     1,     4,
         631,   499,  2021,     4,   486,    15,     3,   179,  1204,
           3,    13,  6898,     1,   950,   234,    41,   214,   981,
          26,     6,    48,  1078,  8892,     2,  3230,     3,    28,
        1015,   205,    47,    70,   223,     8,  2092,  1570,   121,
          45, 15087,    71,   253, 23500,    27,   451,     1,    19,
        9750,    12,   158,     2,    10,     3,     5,    64,    28,
         228,    34,  3548,    11,   529,  1334,     9,   827,  2475,
           2,    17,    14,     5,   378,    27,   165,     1,  3053,
       12169], dtype=int32)

In [13]:
# The vocab_size is given by the number of unique integers in the X_input array
vocab_size = len(np.unique(X_input))

In [14]:
from nbresult import ChallengeResult

result = ChallengeResul
t('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [15]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [16]:
X_input.shape

(29951, 100)

In [17]:
# This function allows to instantiate the model architecture
def build_model():
    
    # We instantiate an object from the Sequential class
    model = Sequential()
    
    # We add the embedding layer with an arbitrarily large input_dim   
    # We map words into a 10-dimensional space 
    model.add(layers.Embedding(input_dim=100000, output_dim=10, input_length=100, mask_zero=True))
    
    # First reccurrent layer returning sequences
    model.add(layers.LSTM(10, activation='tanh', return_sequences=True))
    
    # Second recurrent layer with flat output
    model.add(layers.LSTM(5, activation='tanh', return_sequences=False))
    
    # Fully connected layer 
    model.add(layers.Dense(15, activation='relu'))

    # Fully connected layer 
    model.add(layers.Dense(5, activation='relu'))
    
    # Output layer with sigmoid activation for the binary classification task
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [18]:
# We instantiate the model
model = build_model()

# And have a look at the summary of the architecture 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 10)           1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100, 10)           840       
_________________________________________________________________
lstm_1 (LSTM)                (None, 5)                 320       
_________________________________________________________________
dense (Dense)                (None, 15)                90        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 1,001,336
Trainable params: 1,001,336
Non-trainable params: 0
______________________________________________

In [19]:
# We compile the model with the right loss and an optimizer adapted to a recurrent network
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [20]:
# We instantiate the early stopping criterion
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [21]:
# We fit the model storing the history of the training 
history = model.fit(
    X_input, y,
    validation_split=0.3, 
    epochs=1000, batch_size=32,
    callbacks=[es],
    verbose=1
)

Epoch 1/1000
656/656 [==============================] - 107s 163ms/step - loss: 0.4650 - accuracy: 0.7876 - val_loss: 0.3010 - val_accuracy: 0.8803
Epoch 2/1000
656/656 [==============================] - 101s 155ms/step - loss: 0.2296 - accuracy: 0.9112 - val_loss: 0.2535 - val_accuracy: 0.8971
Epoch 3/1000
656/656 [==============================] - 101s 155ms/step - loss: 0.1766 - accuracy: 0.9330 - val_loss: 0.2775 - val_accuracy: 0.8848
Epoch 4/1000
656/656 [==============================] - 100s 152ms/step - loss: 0.1531 - accuracy: 0.9423 - val_loss: 0.2656 - val_accuracy: 0.9013
Epoch 5/1000
656/656 [==============================] - 100s 152ms/step - loss: 0.1392 - accuracy: 0.9475 - val_loss: 0.3326 - val_accuracy: 0.8901
Epoch 6/1000
656/656 [==============================] - 100s 153ms/step - loss: 0.1280 - accuracy: 0.9528 - val_loss: 0.2730 - val_accuracy: 0.9001
Epoch 7/1000
656/656 [==============================] - 100s 152ms/step - loss: 0.1182 - accuracy: 0.9572 - val_

In [22]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()